In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Load job data
data = pd.read_csv("job_data.csv")
data_cleaned = data.drop(columns=[col for col in data if col.startswith('Unnamed')])
data_cleaned['Salary'] = data_cleaned['Salary'].replace('[\$,]', '', regex=True).astype(float)

In [3]:
# Load Zillow data
zillow_data = pd.read_csv("Zillow_Data.csv")

In [4]:
# Load median salary data
median_salary_state = pd.read_csv("Median Salary by State.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Median Salary by State.csv'

In [ ]:
# Convert the Median Rent from string to float and remove formatting
zillow_data['Median Rent'] = zillow_data['Median Rent'].replace({'\$':'', ',':''}, regex=True).astype(float)

In [ ]:
# Convert Median Salary to a float and remove any dollar signs and commas
median_salary_state['Median Salary'] = median_salary_state['Median Salary'].replace('[\$,]', '', regex=True).astype(float)

In [5]:
# Calculate Annual Median Rent
zillow_data['Annual Median Rent'] = zillow_data['Median Rent'] * 12

In [6]:
# Calculate Salary Needed
zillow_data['Salary Needed'] = zillow_data['Annual Median Rent'] * 3.3333

In [25]:
# Create function to calculate the average salary
def calculate_average_salary(company, job_tag, state):
    filtered_data = data_cleaned[
        (data_cleaned['Company'] == company) & 
        (data_cleaned['Job Tag'] == job_tag) & 
        (data_cleaned['State'] == state)
    ]
    average_salary = filtered_data['Salary'].mean()
    return average_salary

In [26]:
# Create function to compare the average salary to the salary needed
def compare_salary_to_cost_of_living(average_salary, state, zillow_data):
    # Get the 'Salary Needed' for the state from zillow_data
    salary_needed = zillow_data.loc[zillow_data['State'] == state, 'Salary Needed'].values[0]
    return average_salary >= salary_needed

In [27]:
# Create dropdowns
company_widget = widgets.Dropdown(options=data_cleaned['Company'].unique(), description='Company:')
job_tag_widget = widgets.Dropdown(options=data_cleaned['Job Tag'].unique(), description='Job Tag:')
state_widget = widgets.Dropdown(options=data_cleaned['State'].unique(), description='State:')

In [28]:
# Button to calculate the average salary
calculate_button = widgets.Button(description="Calculate Average Salary")

In [29]:
# Output widget to display the result
output = widgets.Output()

In [30]:
# Function for button click
def on_calculate_clicked(b):
    with output:
        output.clear_output()
        average_salary = calculate_average_salary(company_widget.value, job_tag_widget.value, state_widget.value)
        if pd.isna(average_salary):
            print("No data available for the selected criteria.")
        else:
            # Median salary for the selected state
            median_salary_row = median_salary_state[median_salary_state['State'] == state_widget.value]
            if not median_salary_row.empty:
                median_salary = median_salary_row['Median Salary'].values[0]
                salary_difference = average_salary - median_salary

                comparison_result = compare_salary_to_cost_of_living(average_salary, state_widget.value, zillow_data)
                print(f"The average salary for your selections is: ${average_salary:.2f}")
                print(f"Is the average salary greater than the salary needed in {state_widget.value}? {'Yes' if comparison_result else 'No'}")
                print(f"The average salary is ${salary_difference:.2f} more than the median salary in {state_widget.value}.")
            else:
                print("Median salary data for the selected state is not available.")

In [31]:
# Link the button click event to the function
calculate_button.on_click(on_calculate_clicked)

In [32]:
# Display the widgets
display(widgets.VBox([company_widget, job_tag_widget, state_widget, calculate_button, output]))

In [105]:
# Extract uodated Zillow data as CSV
#output_file_path = "updated_zillow_data.csv"
#zillow_data.to_csv(output_file_path, index=False)